In [11]:
import geopandas as gpd
import pandas as pd
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib.patches as plt_patch
%matplotlib inline
import shapely.geometry as geom
from descartes import PolygonPatch
from shapely.geometry import Point, LineString
import folium
import tqdm
import joblib

In [54]:
def clean_data(geodf):
    
    # Finds midpoint of geometry string
    def midpoint(geometry):
        midpoint = geometry.interpolate(0.5, normalized=True)
        return midpoint
    
    def intersectionpoint1(geometry):
        intersec_point_1 = geometry.interpolate(0, normalized=True)
        return intersec_point_1

    def intersectionpoint2(geometry):
        intersec_point_2 = geometry.interpolate(1, normalized=True)
        return intersec_point_2

    
    geodf['midpoint'] = midpoint(geodf.geometry) #geometry:column with Linestring
    geodf['intersec_point_1'] = intersectionpoint1(geodf.geometry)
    geodf['intersec_point_2'] = intersectionpoint2(geodf.geometry)
    geodf['lon_midpoint'] = geodf.midpoint.apply(lambda p: p.x)
    geodf['lat_midpoint'] = geodf.midpoint.apply(lambda p: p.y)
    geodf['lon_interpoint_1'] = geodf.intersec_point_1.apply(lambda p: p.x)
    geodf['lat_interpoint_1'] = geodf.intersec_point_1.apply(lambda p: p.y)
    geodf['lon_interpoint_2'] = geodf.intersec_point_2.apply(lambda p: p.x)
    geodf['lat_interpoint_2'] = geodf.intersec_point_2.apply(lambda p: p.y)
    
    def create_list_coord(column_coord_lon,column_coord_lat):
        list_point = []
        M=[]
        for k in range(len(geodf)):
            M=[]
            a = column_coord_lon
            b = column_coord_lat
            M.append(a[k]) 
            M.append(b[k]) 
            list_point.append(M)
        return list_point

    
    
    list_point_mid = create_list_coord(geodf['lon_midpoint'],geodf['lat_midpoint'])
    list_point_inter_1 = create_list_coord(geodf['lon_interpoint_1'],geodf['lat_interpoint_1'])
    list_point_inter_2 = create_list_coord(geodf['lon_interpoint_2'],geodf['lat_interpoint_2'])
    list_global_points = list_point_mid + list_point_inter_1 + list_point_inter_2 
    
    def points_dict(somelist):
        return {str(somelist.index(x)) : x for x in somelist}
    
    list_global_unique = []
    for ele in list_global_points: 
        if ele not in list_global_unique: 
            list_global_unique.append(ele)
    mapper_points_dict = points_dict(list_global_unique)
    return mapper_points_dict

In [114]:
def from_cleaning_to_end(geodf):
    
    def key_identifier(lon,lat):
        for key, value in zip(mapper_points_dict.keys(), mapper_points_dict.values()):
            if [lon, lat] == value:
                return(key)
    
    trc = geodf[['lon_midpoint', 'lat_midpoint','lon_interpoint_1', 'lat_interpoint_1', 'lon_interpoint_2','lat_interpoint_2']]
    dic = dict()
    for x, y, x_1, y_1, x_2, y_2 in tqdm.tqdm(trc.values):
        dic[key_identifier(x, y)] = {str(key_identifier(x_1, y_1)): 1, str(key_identifier(x_2, y_2)): 1}
        
    lst =[]
    for value in tqdm.tqdm(mapper_points_dict.values()):
        lst.append(key_identifier(value[0], value[1]))
    lst = np.unique(lst)
    
    tmp = dict()
    for key in tqdm.tqdm(lst):
        tmp[key] = dict()
        for entry in dic.keys():
            if key in dic[entry].keys():
                tmp[key].update({str(entry) :1})
                
    new = dict()
    for key in dic.keys(): new[key] = dic[key]
    for key in tmp.keys(): 
        if key not in new.keys(): 
            new[key] = tmp[key]
            
    joblib.dump(new, 'map_sf.jb')
    joblib.dump(mapper_points_dict, 'mapper_points.jb')
    return new['10000']

In [108]:
def path_to_points(data_points):
    P =[]
    for k in range(len(data_point)):
        for key, value in zip(mapper_points_dict.keys(), mapper_points_dict.values()):
            if data_point[k] == key:
                P.append(value)
    traj = gpd.GeoDataFrame(columns=['ID', 'LON', 'LAT'])
    traj.loc[:,'ID'] = data_point[:]

    for k in range(len(P)):
        traj.loc[k,'LON'] = P[:][k][0]
        traj.loc[k,'LAT'] = P[:][k][1]
    traj = gpd.GeoDataFrame(traj, geometry=[Point(x, y) for x, y in zip(traj.LON, traj.LAT)])
    return traj

In [109]:
geodf_sf = gpd.GeoDataFrame.from_file("geo_export_23295268-4675-4bf2-a0fd-5f267e42670c.shp")

In [116]:
mapper_points_dict = clean_data(geodf_sf)

In [117]:
from_cleaning_to_end(geodf_sf)

100%|███████████████████████████████████████████████████████████████████████████| 25956/25956 [01:10<00:00, 370.61it/s]


{'23268': 1, '23324': 1}

In [118]:
data_point = ['300', '16504', '7866', '17397', '1606', '21464', '6804', '17506',
              '1730', '17507', '1731', '17508', '3765', '17412', '13947', '21062',
              '6159', '17411', '12575', '20394', '12576', '20023', '12577', '24691',
              '13522', '19149', '3659', '19148', '4225', '16552', '371', '16550', '368',
              '16549', '1114', '17118', '1112', '17117', '1109', '17114', '1107', '17112',
              '989', '17028', '1006', '17037', '1005', '17033', '1004', '17036', '1003',
              '17032', '1002', '17030', '1001', '17031', '1000']

In [119]:
traj = path_to_points(data_point)
traj

,ID,LON,LAT,geometry
0,300,-122.393,37.7287,POINT (-122.3927743531825 37.72865581199223)
1,16504,-122.393,37.7285,POINT (-122.3928994196241 37.72848958051349)
2,7866,-122.393,37.7282,POINT (-122.3932346246624 37.72823951718651)
3,17397,-122.394,37.7279,POINT (-122.3935288874744 37.72791331272369)
4,1606,-122.394,37.7285,POINT (-122.3944834149079 37.72845496837299)
5,21464,-122.395,37.729,POINT (-122.3954379423414 37.72899662402229)
6,6804,-122.395,37.7293,POINT (-122.395161839743 37.72929053493576)
7,17506,-122.395,37.7296,POINT (-122.3948857371446 37.72958444584922)
8,1730,-122.395,37.7296,POINT (-122.3951313349957 37.72961933324713)
9,17507,-122.395,37.7297,POINT (-122.3953769328468 37.72965422064504)


In [171]:
def gen_traj(traj):
    import geopandas as gpd
    import pandas as pd
    import numpy as np
    from mpl_toolkits.basemap import Basemap
    import matplotlib.pyplot as plt
    import matplotlib.patches as plt_patch
    %matplotlib inline
    import shapely.geometry as geom
    from descartes import PolygonPatch
    from shapely.geometry import Point, LineString
    import folium
    import tqdm
    import joblib
    
    geom0 = traj.loc[0]['geometry']
    geom1 = traj.loc[1]['geometry']

    # Create LineString from coordinates
    start, end = [(geom0.x, geom0.y), (geom1.x, geom1.y)]
    line = LineString([start, end])
    print(f"Geometry type: {str(type(line))}")
    line

    def make_lines(gdf, df_out, i, geometry = 'geometry'):
        geom0 = gdf.loc[i][geometry]
        geom1 = gdf.loc[i + 1][geometry]

        start, end = [(geom0.x, geom0.y), (geom1.x, geom1.y)]
        line = LineString([start, end])

        # Create a DataFrame to hold record
        data = {'id': i,
                'geometry': [line]}
        df_line = pd.DataFrame(data, columns = ['id', 'geometry'])

        # Add record DataFrame of compiled records
        df_out = pd.concat([df_out, df_line])
        return df_out

    def add_markers(mapobj, gdf):
        coords = []
        for i, row in gdf.iterrows():
            coords.append([row.geometry.y, row.geometry.x])
        for coord in coords:
            folium.CircleMarker(location = coord,
                                radius = 2.5, 
                                fill = True,
                                fill_color = '#F50057',
                                fill_opacity = 0.75,
                                color = 'whitesmoke',
                                weight = 0.5).add_to(mapobj)
        return mapobj

    f = folium.Figure(height = 600)
    m = folium.Map([37.76, -122.44], zoom_start = 12, tiles='Cartodb dark_matter')
    m.add_to(f)
    #add_markers(m, traj)


    # initialize an output DataFrame
    df = pd.DataFrame(columns = ['id', 'geometry'])

    # Loop through each row of the input point GeoDataFrame
    x = 0
    while x < len(traj) - 1:
        df = make_lines(traj, df, x)
        x = x + 1

    #df.head()


    crs = {'init': 'epsg:4326'}
    gdf_line = gpd.GeoDataFrame(df, crs=crs)
    #gdf_line.head()


    return gdf_line

In [173]:
gen_traj(traj)

Geometry type: <class 'shapely.geometry.linestring.LineString'>


,id,geometry
0,0,LINESTRING (-122.3927743531825 37.728655811992...
0,1,LINESTRING (-122.3928994196241 37.728489580513...
0,2,LINESTRING (-122.3932346246624 37.728239517186...
0,3,LINESTRING (-122.3935288874744 37.727913312723...
0,4,LINESTRING (-122.3944834149079 37.728454968372...
0,5,LINESTRING (-122.3954379423414 37.728996624022...
0,6,LINESTRING (-122.395161839743 37.7292905349357...
0,7,LINESTRING (-122.3948857371446 37.729584445849...
0,8,LINESTRING (-122.3951313349957 37.729619333247...
0,9,LINESTRING (-122.3953769328468 37.729654220645...


In [172]:
f = folium.Figure(height = 600)
m = folium.Map([37.76, -122.44], zoom_start = 12, tiles='Cartodb dark_matter')
m.add_to(f)
a = gen_traj(traj)
a.add_to(m)
m

Geometry type: <class 'shapely.geometry.linestring.LineString'>


AttributeError: 'GeoDataFrame' object has no attribute 'add_to'

In [165]:
def trajectory(list_ids, filepath='/mapper_points.jb'):
    
    mapper_points_dict = joblib.load(filepath)
    
    def path_to_points(data_points):
        P =[]
        for k in range(len(data_points)):
            for key, value in zip(mapper_points_dict.keys(), mapper_points_dict.values()):
                if data_points[k] == key:
                    P.append(value)
        traj = gpd.GeoDataFrame(columns=['ID', 'LON', 'LAT'])
        traj.loc[:,'ID'] = data_points[:]

        for k in range(len(P)):
            traj.loc[k,'LON'] = P[:][k][0]
            traj.loc[k,'LAT'] = P[:][k][1]
        traj = gpd.GeoDataFrame(traj, geometry=[Point(x, y) for x, y in zip(traj.LON, traj.LAT)])
        return traj
    
    traj = path_to_points(list_ids)
    
    def gen_traj(traj):
        geom0 = traj.loc[0]['geometry']
        geom1 = traj.loc[1]['geometry']

        # Create LineString from coordinates
        start, end = [(geom0.x, geom0.y), (geom1.x, geom1.y)]
        line = LineString([start, end])
        print(f"Geometry type: {str(type(line))}")
        line

        def make_lines(gdf, df_out, i, geometry = 'geometry'):
            geom0 = gdf.loc[i][geometry]
            geom1 = gdf.loc[i + 1][geometry]

            start, end = [(geom0.x, geom0.y), (geom1.x, geom1.y)]
            line = LineString([start, end])

            # Create a DataFrame to hold record
            data = {'id': i,
                    'geometry': [line]}
            df_line = pd.DataFrame(data, columns = ['id', 'geometry'])

            # Add record DataFrame of compiled records
            df_out = pd.concat([df_out, df_line])
            return df_out

        def add_markers(mapobj, gdf):
            coords = []
            for i, row in gdf.iterrows():
                coords.append([row.geometry.y, row.geometry.x])
            for coord in coords:
                folium.CircleMarker(location = coord,
                                    radius = 2.5, 
                                    fill = True,
                                    fill_color = '#F50057',
                                    fill_opacity = 0.75,
                                    color = 'whitesmoke',
                                    weight = 0.5).add_to(mapobj)
            return mapobj


        # initialize an output DataFrame
        df = pd.DataFrame(columns = ['id', 'geometry'])

        # Loop through each row of the input point GeoDataFrame
        x = 0
        while x < len(traj) - 1:
            df = make_lines(traj, df, x)
            x = x + 1

        #df.head()


        crs = {'init': 'epsg:4326'}
        gdf_line = gpd.GeoDataFrame(df, crs=crs)
        #gdf_line.head()


        return folium.GeoJson(gdf_line)
    return gen_traj(traj)

In [166]:
a = trajectory(data_point,'mapper_points.jb')

Geometry type: <class 'shapely.geometry.linestring.LineString'>


In [167]:
f = folium.Figure(height = 600)
m = folium.Map([37.76, -122.44], zoom_start = 12, tiles='Cartodb dark_matter')
m.add_to(f)
a.add_to(m)
m

In [168]:
data_points_1 = ['40', '16274', '16180', '16281', '47', '16271', '48', '16268', '34', '16267', '32', '22627', '11215', '22762', '11214', '23662', '10887', '23556', '10888', '23090', '10889', '22906', '9264', '22687', '8888', '22686', '12920', '24526', '12921', '22491', '8558', '22490', '12613', '21508', '12614', '16465', '254', '16464', '252', '16463', '250', '16462', '249', '16461', '15030', '18601', '15031', '21416', '15032', '17088', '1066', '17092', '1072', '17080', '1071', '17091', '1070', '17081', '1059', '17046', '8185', '16683', '8184', '16432', '8182', '22245', '8180', '18503', '2969', '18590', '2971', '18591', '2974', '16411', '2976', '18545', '2906', '18546', '2908', '18547', '2911', '17743', '2913', '18549', '2915', '18550', '2917', '18551', '7449', '19447', '7450', '21241', '6422', '21240', '70']

In [169]:
b = trajectory(data_points_1,'mapper_points.jb')

Geometry type: <class 'shapely.geometry.linestring.LineString'>


In [170]:
f = folium.Figure(height = 600)
m = folium.Map([37.76, -122.44], zoom_start = 12, tiles='Cartodb dark_matter')
m.add_to(f)
a.add_to(m)
b.add_to(m)
m